# Processing mixed beverage data

## December 2017 sales report

This Jupyter Notebook uses data.texas.gov [Mixed Beverage Gross Receipts data](https://data.texas.gov/Government-and-Taxes/Mixed-Beverage-Gross-Receipts/naix-2893), and then a python library called [agate](http://agate.readthedocs.io/) to clean and process that data for [stories similar to this one](http://www.mystatesman.com/business/austin-alcohol-sales-percent-february/Oo2txZUkuDlqBl0rU9O1lJ/) on monthly alcohol sales.

There are two places where **last day of the month** has to be entered into the script, in YYYY-MM-DD format:
- `download_date` in the "Download" section
- `date_studied` in the "Setting up" section

For December 2017, use:

2017-12-31

## Download

We're using the bash curl command to download our file from Socrata.

### Set download date

Make sure to set the `download_date` below to the last day of the month studied.

In [1]:
%%bash
download_date='2017-12-31'

socrata_data='https://data.texas.gov/resource/fp9t-htqh.json?'
socrata_limit='$limit=20000'
socrata_date='&$where=obligation_end_date_yyyymmdd%20=%20%27'$download_date'%27'
socrata_path=$socrata_data$socrata_limit$socrata_date

echo $socrata_path

curl -L -o ../mixbev-files/Mixed_Beverage_Gross_Receipts_$download_date.json \
$socrata_path

https://data.texas.gov/resource/fp9t-htqh.json?$limit=20000&$where=obligation_end_date_yyyymmdd%20=%20%272017-12-31%27


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.8M    0 10.8M    0     0   667k      0 --:--:--  0:00:16 --:--:--  741k


## Set date_studied

It is imporant to change `date_studied` to the last day of the month studied:

In [2]:
# this should be the same as the download_date above.
date_studied = '2017-12-31'

## Set up python

We'll import the python libraries we need, and then set the date_studied, which is important to find the right file.

In [3]:
# import libraries
import agate
# this surpresses the timezone warning
# Might comment out during development so other warnings
# are not surpressed
import warnings
warnings.filterwarnings('ignore')

## Import the file

We set the path name based on `date_studied` above. We import the raw data, forcing some data types.

In [4]:
# This is the source file, which should be donwloaded
file_path = '../mixbev-files/Mixed_Beverage_Gross_Receipts_' + date_studied + '.json'

# Setting up to force column types for some columns
specified_types = {
    'cover_charge_receipts': agate.Number(),
    'taxpayer_number': agate.Text(),
    'taxpayer_zip': agate.Text(),
    'taxpayer_county': agate.Text(),
    'location_number': agate.Text(),
    'location_address': agate.Text(),
    'location_zip': agate.Text(),
    'location_county': agate.Text(),
}

# this imports the file specified above, along with the proper types
mixbev_raw = agate.Table.from_json(file_path, column_types=specified_types)

# Uncomment below to peek at column names
# print(mixbev_raw)

## Cleaning bad columns

There is a problem with the Comptroller data where two columns are incorrectly labeled. They are NOT used in the analysis, but we'll fix them anyway. It could be that the Comptroller will fix these in the future.

In [5]:
# renames two columns that are labelled incorrectly from original data
mixbev_fixed = mixbev_raw.rename(column_names = {
        'tabc_permit_number': 'inside_outside_city_limits_code',
        'inside_outside_city_limits_code_y_n': 'tabc_permit',
    }
)

# Uncomment below to see changed columns
# print(mixbev_fixed)

In [6]:
# establish our table of record
mixbev = mixbev_fixed

## Create establishment column

We do this so we make sure we have single establishments instead of grouping trade names together from different addresses, like 'CHILI'S BAR & GRILL'.

In [7]:
# Concatenates the name and address
mixbev_establishment = mixbev.compute([
    ('Establishment', agate.Formula(agate.Text(), lambda row: '%(location_name)s (%(location_address)s)' % row))
])

# Uncomment line below to print Establishment to check what is looks like
# mixbev_establishment.select('Establishment').limit(5).print_table(max_column_width=80)

In [8]:
# Uncomment below to see column names
# print(mixbev_establishment)

## Import and merge counties lookup table

We do this to get county names. I got this list from the comptroller.

NOTE: Wisdom would suggest we join on the code column from counites, but the data.texas.gov data does not have the zero padding from those values, so I'm using the id column.

In [9]:
# importing countes.csv, ensuring that the 'code' column is text
counties = agate.Table.from_csv('../resource-files/counties.csv', column_types={
    'code': agate.Text(),
    'id': agate.Text()
})

# uncomment below to peek at the column names and an example
# print(counties)
# counties.limit(5).print_table()

# joines the counties table to the mixed bev cleaned data with establishments
mixbev_joined = mixbev_establishment.join(counties, 'location_county', 'id')

# uncomment below to see merged column names 
# print(mixbev_joined)

In [10]:
# get just the columns we need and rename county
# THIS is the finished, cleaned mixbev table
mixbev = mixbev_joined.select([
    'location_name',
    'location_address',
    'Establishment',
    'location_city',
    'location_state',
    'location_zip',
    'county',
    'total_receipts',
    'obligation_end_date_yyyymmdd'
]).rename(column_names = {
    'location_name' : 'Name',
    'location_address' : 'Address',
    'location_city': 'City',
    'location_state': 'State',
    'location_zip': 'Zip',
    'total_receipts' : 'Receipts',
    'county': 'County',
    'obligation_end_date_yyyymmdd': 'Report date'
})

# peek at the column names
print(mixbev)

| column        | data_type |
| ------------- | --------- |
| Name          | Text      |
| Address       | Text      |
| Establishment | Text      |
| City          | Text      |
| State         | Text      |
| Zip           | Text      |
| County        | Text      |
| Receipts      | Number    |
| Report date   | DateTime  |



## Location sums function

Because we want to get the top sellers in a bunch of cities and couties, we create a function so we don't have to repeat the code. This function allows us to pass in a city or county name to filter the monthly receipts table and then sum the Tax and Receipts columns. The result can then be acted on to print or aggreggate. It is used later in the file.

In [11]:
# function to group sales by a specific location
# City or County passed in should be ALL CAPS
# Location_type can be 'City' or 'County'

def location_sum(location_type, location):
    # Filters the data to the specified city
    location_filtered = mixbev.where(lambda row: row[location_type].upper() == location)

    # groups the data based on Establishment and location
    location_grouped = location_filtered.group_by('Establishment').group_by(location_type)
    # computes the sales based on the grouping
    location_summary = location_grouped.aggregate([
        ('Receipts_sum', agate.Sum('Receipts'))
    ])
    
    # sorts the results by most sold
    location_summary_sorted = location_summary.order_by('Receipts_sum', reverse=True)
    # prints the top 10 results
    
    return(location_summary_sorted)

## Data checking
I make sure I'm looking the correct month, and to make sure the number of records is not hitting my download limit.

In [12]:
# double-checking I'm looking at one month of data
mixbev_dates = mixbev.select('Report date').distinct('Report date')
print('This report date should equal your date_studied:\n')
mixbev_dates.print_table()
print('\nIf the number of records below is 20000, increase the download limit.')
print('\nNumber of records in table: {}'.format(
        len(mixbev))
     )

This report date should equal your date_studied:

|         Report date |
| ------------------- |
| 2017-12-31 00:00:00 |

If the number of records below is 20000, increase the download limit.

Number of records in table: 15585


## Top sales statewide

Because we want to group our results by more than one field and perform more than one aggregation, we'll do this a little differently. We'll use group_by to create a grouped table, then perform aggregations on that new table to compute the Tax and Receipts columns.

In [13]:
# summing sales statewide for month

print('Total statewide sales for this month are: {}\n'.format(
    mixbev.aggregate(agate.Sum('Receipts')) # <<< I should format the number better
))

# groups the data based on Establishment and City
mixbev_grouped = mixbev.group_by('Establishment').group_by('County').group_by('City')

# computes the sales based on the grouping
state_summary = mixbev_grouped.aggregate([
    ('Sales_sum', agate.Sum('Receipts'))
])

# sorts the results by most sold. We could probalby chain it above if we wanted to.
state_summary_sorted = state_summary.order_by('Sales_sum', reverse=True)

# prints the top 10 results
state_summary_sorted.limit(10).print_table(max_column_width=40)


Total statewide sales for this month are: 618477471

| Establishment                            | County     | City        | Sales_sum |
| ---------------------------------------- | ---------- | ----------- | --------- |
| AT&T STADIUM (1 LEGENDS WAY)             | Tarrant    | ARLINGTON   | 3,983,415 |
| GAYLORD TEXAN (1501 GAYLORD TRL)         | Tarrant    | GRAPEVINE   | 2,063,706 |
| LEVY RESTAURANTS AT TOYOTA CENTER (15... | Harris     | HOUSTON     | 1,506,249 |
| LEVY RESTAURANTS (2500 VICTORY AVE)      | Dallas     | DALLAS      | 1,330,793 |
| BOTTLED BLONDE- THE BACK YARD (501 N ... | Dallas     | DALLAS      | 1,154,213 |
| ARAMARK SPORTS AND ENTERTAINMENT SERV... | Bexar      | SAN ANTONIO | 1,046,692 |
| SPECTRUM CATERING (27328 SPECTRUM WAY)   | Montgomery | CONROE      | 1,010,510 |
| HOSPITALITY INTERNATIONAL, INC. (2380... | Bexar      | SAN ANTONIO | 1,001,804 |
| SALC, INC. (2201 N STEMMONS FWY FL 1)    | Dallas     | DALLAS      |   939,518 |
| BILLY BOB'S TEXAS (25

## Austin sales and sums

With this, we reference the location_sum function above, and pass the type of location (City) and the name of the city (AUSTIN). At the same time, we limit number of recoreds in the return, and then print the results. We are basically stringing together a bunch of stuff at once.

In [14]:
# Austin total sales as s city
# This sums the grouped table, but it works

print('Total sales in Austin are: {}\n'.format(
    location_sum('City', 'AUSTIN').aggregate(agate.Sum('Receipts_sum'))
))

# uses the city_sum function to filter
austin = location_sum('City', 'AUSTIN')


# print the resulting table
print('Top sellers in Austin are:\n')
austin.limit(25).print_table(max_column_width=60, max_rows=None)

Total sales in Austin are: 64057149

Top sellers in Austin are:

| Establishment                                                | City   | Receipts_sum |
| ------------------------------------------------------------ | ------ | ------------ |
| WLS BEVERAGE CO (110 E 2ND ST)                               | AUSTIN |      869,818 |
| W HOTEL AUSTIN (200 LAVACA ST)                               | AUSTIN |      581,798 |
| DH BEVERAGE LLC (604 BRAZOS ST)                              | AUSTIN |      563,253 |
| ROSE ROOM/ 77 DEGREES (11500 ROCK ROSE AVE)                  | AUSTIN |      527,145 |
| 400 BAR/CUCARACHA/CHUPACABRA/JACKALOPE/MOOSENUCKLE (400 E... | AUSTIN |      466,017 |
| SAN JACINTO BEVERAGE COMPANY LLC (98 SAN JACINTO BLVD)       | AUSTIN |      459,049 |
| THE ROOSEVELT ROOM & THE ELEANOR (307 W 5TH ST # 307B)       | AUSTIN |      436,238 |
| THE DOGWOOD DOMAIN (11420 ROCK ROSE AVE STE 700)             | AUSTIN |      419,467 |
| KUNG FU SALOON (11501 ROCK ROSE AVE STE 140

## More Central Texas cities
More of the same for different cities.

In [15]:
location_sum('City', 'BASTROP').limit(5).print_table(max_column_width=60)

| Establishment                                                | City    | Receipts_sum |
| ------------------------------------------------------------ | ------- | ------------ |
| OLD TOWN RESTURANT AND BAR/PINEY CREEK CHOP HOUSE (931 MA... | BASTROP |       94,519 |
| SP BASTROP THEATRE, LP (1600 CHESTNUT ST)                    | BASTROP |       43,273 |
| CHILI'S GRILL & BAR (734 HIGHWAY 71 W)                       | BASTROP |       42,059 |
| BACK 9 (834 HIGHWAY 71 W)                                    | BASTROP |       27,018 |
| NEIGHBOR'S (601 CHESTNUT ST UNIT C)                          | BASTROP |       26,995 |


In [16]:
location_sum('City', 'BEE CAVE').limit(3).print_table(max_column_width=60)

| Establishment                                         | City     | Receipts_sum |
| ----------------------------------------------------- | -------- | ------------ |
| WOODY TAVERN AND GRILL, INC. (12801 SHOPS PKWY # 100) | BEE CAVE |      109,891 |
| MAUDIE'S HILL COUNTRY, LLC (12506 SHOPS PKWY)         | BEE CAVE |       84,090 |
| HCG BEVERAGE, LLC (12525 BEE CAVE PKWY)               | BEE CAVE |       76,546 |


In [17]:
location_sum('City', 'BUDA').limit(3).print_table(max_column_width=60)

| Establishment                                               | City | Receipts_sum |
| ----------------------------------------------------------- | ---- | ------------ |
| BUCKS BACKYARD (1750 S FM 1626)                             | BUDA |       94,575 |
| NATE'S AT BUDA MILL AND GRAIN, INC. (306 S MAIN ST STE 101) | BUDA |       70,573 |
| PINBALLZ KINGDOM (15201 S IH 35)                            | BUDA |       54,395 |


In [18]:
location_sum('City', 'DEL VALLE').limit(3).print_table(max_column_width=60)

| Establishment                                                | City      | Receipts_sum |
| ------------------------------------------------------------ | --------- | ------------ |
| RYAN SANDERS SPORTS SERVICES, LLC (9201 CIRCUIT OF THE AM... | DEL VALLE |      112,913 |
| CLUB MIS AMIGOS (13820 FM 812)                               | DEL VALLE |        7,849 |
| RESTAURANTES JASMINES (2463 HIGWAY 71 E)                     | DEL VALLE |        2,975 |


In [19]:
location_sum('City', 'CEDAR PARK').limit(3).print_table(max_column_width=60)

| Establishment                                                | City       | Receipts_sum |
| ------------------------------------------------------------ | ---------- | ------------ |
| CHUY'S (4911 183A TOLL RD)                                   | CEDAR PARK |      172,579 |
| THE GROVE WINE BAR AND KITCHEN (1310 E WHITESTONE BLVD # ... | CEDAR PARK |      128,384 |
| LUPE TORTILLA MEXICAN RESTAURANT (4501 183A TOLL RD STE B)   | CEDAR PARK |      113,756 |


In [20]:
location_sum('City', 'DRIPPING SPRINGS').limit(3).print_table(max_column_width=60)

| Establishment                                         | City             | Receipts_sum |
| ----------------------------------------------------- | ---------------- | ------------ |
| TRUDY'S FOUR STAR (13059 FOUR STAR BLVD)              | DRIPPING SPRINGS |       58,757 |
| FLORES MEXICAN RESTAURANT (2440 E HIGHWAY 290 BLDG D) | DRIPPING SPRINGS |       49,108 |
| DEEP EDDY DISTILLING CO (2250 E HIGHWAY 290)          | DRIPPING SPRINGS |       40,141 |


In [21]:
location_sum('City', 'GEORGETOWN').limit(3).print_table(max_column_width=60)

| Establishment                              | City       | Receipts_sum |
| ------------------------------------------ | ---------- | ------------ |
| GATEWAY BEVERAGES, INC (1101 WOODLAWN AVE) | GEORGETOWN |      102,240 |
| EL MONUMENTO (205 W 2ND ST)                | GEORGETOWN |       79,931 |
| DOS SALSAS CAFE INC (1104 S MAIN ST)       | GEORGETOWN |       66,811 |


In [22]:
location_sum('City', 'KYLE').limit(3).print_table(max_column_width=60)

| Establishment                                           | City | Receipts_sum |
| ------------------------------------------------------- | ---- | ------------ |
| EVO ENTERTAINMENT CENTER (3200 KYLE XING)               | KYLE |       69,790 |
| CASA GARCIA'S MEXICAN RESTAURANT (5401 FM 1626 STE 300) | KYLE |       69,438 |
| CENTERFIELD SPORTS BAR & GRILL (200 W CENTER ST)        | KYLE |       39,396 |


In [23]:
location_sum('City', 'LAGO VISTA').limit(3).print_table(max_column_width=60)

| Establishment                                         | City       | Receipts_sum |
| ----------------------------------------------------- | ---------- | ------------ |
| THE GRILLE AT HIGHLAND LAKES (20552 HIGHLAND LAKE DR) | LAGO VISTA |       21,706 |
| THE GNARLY GAR (18200 LAKEPOINT CV)                   | LAGO VISTA |       20,760 |
| COPPERHEAD GRILL (6115 LOHMANS FORD RD)               | LAGO VISTA |       15,490 |


In [24]:
location_sum('City', 'LAKEWAY').limit(3).print_table(max_column_width=60)

| Establishment                                          | City    | Receipts_sum |
| ------------------------------------------------------ | ------- | ------------ |
| THE GROVE WINE BAR AND KITCHEN (3001 RANCH ROAD 620 S) | LAKEWAY |      106,006 |
| HIGH 5 ENTERTAINMENT (1502 RANCH ROAD 620 S)           | LAKEWAY |       99,202 |
| MCARTHUR'S (1516 RANCH ROAD 620 S STE F200)            | LAKEWAY |       59,008 |


In [25]:
location_sum('City', 'LEANDER').limit(3).print_table(max_column_width=60)

| Establishment                                          | City    | Receipts_sum |
| ------------------------------------------------------ | ------- | ------------ |
| BROOKLYN HEIGHTS PIZZERIA (3550 LAKELINE BLVD STE 135) | LEANDER |       65,014 |
| MARIA'S BAR AND GRILL (109 N HIGHWAY 183)              | LEANDER |       44,776 |
| JARDIN DEL REY (703 S HIGHWAY 183)                     | LEANDER |       36,470 |


In [26]:
location_sum('City', 'LIBERTY HILL').limit(3).print_table(max_column_width=60)

| Establishment                                        | City         | Receipts_sum |
| ---------------------------------------------------- | ------------ | ------------ |
| JARDIN CORONA (15395 W STATE HIGHWAY 29)             | LIBERTY HILL |       41,526 |
| MARGARITA'S RESTAURANT (10280 W STATE HIGHWAY 29)    | LIBERTY HILL |       34,840 |
| ELENAS MEXICAN RESTAURANT (14801 W STATE HIGHWAY 29) | LIBERTY HILL |        4,582 |


In [27]:
location_sum('City', 'PFLUGERVILLE').limit(3).print_table(max_column_width=60)

| Establishment                                  | City         | Receipts_sum |
| ---------------------------------------------- | ------------ | ------------ |
| MAVERICKS (1700 GRAND AVENUE PKWY STE 240)     | PFLUGERVILLE |      147,058 |
| HANOVER'S DRAUGHT HAUS (108 E MAIN ST)         | PFLUGERVILLE |       66,170 |
| RIO GRANDE TEX MEX (15821 CENTRAL COMMERCE DR) | PFLUGERVILLE |       59,576 |


In [28]:
location_sum('City', 'ROUND ROCK').limit(5).print_table(max_column_width=60)

| Establishment                                             | City       | Receipts_sum |
| --------------------------------------------------------- | ---------- | ------------ |
| CHUY'S ROUND ROCK (2320 N INTERSTATE 35)                  | ROUND ROCK |      165,153 |
| JACK ALLEN'S KITCHEN (2500 HOPPE TRL)                     | ROUND ROCK |      148,393 |
| THIRD BASE ROUND ROCK, LLC (3107 S INTERSTATE 35 STE 810) | ROUND ROCK |      137,628 |
| RICK'S CABARET (3105 S INTERSTATE 35)                     | ROUND ROCK |      137,563 |
| TWIN PEAKS RESTAURANT (100 LOUIS HENNA BLVD)              | ROUND ROCK |      134,566 |


In [29]:
location_sum('City', 'SAN MARCOS').limit(5).print_table(max_column_width=60)

| Establishment                                              | City       | Receipts_sum |
| ---------------------------------------------------------- | ---------- | ------------ |
| THE MARC (120 E SAN ANTONIO ST)                            | SAN MARCOS |      116,365 |
| PLUCKERS WING BAR (105 N INTERSTATE 35)                    | SAN MARCOS |      106,325 |
| 54TH STREET RESTAURANT & DRAFTHOUSE (1303 S INTERSTATE 35) | SAN MARCOS |       99,947 |
| MAYLOO'S (138 N LBJ DR)                                    | SAN MARCOS |       97,476 |
| CHUY'S SAN MARCOS (1121 N INTERSTATE 35)                   | SAN MARCOS |       96,926 |


In [30]:
location_sum('City', 'SPICEWOOD').limit(3).print_table(max_column_width=60)

| Establishment                                        | City      | Receipts_sum |
| ---------------------------------------------------- | --------- | ------------ |
| POODIES HILLTOP ROADHOUSE (22308 STATE HIGHWAY 71 W) | SPICEWOOD |       47,063 |
| ANGEL'S ICEHOUSE (21815 W HWY 71)                    | SPICEWOOD |       35,957 |
| APIS RESTAURANT (23526 STATE HIGHWAY 71 W)           | SPICEWOOD |       28,707 |


In [31]:
location_sum('City', 'SUNSET VALLEY').limit(3).print_table(max_column_width=60)

| Establishment                                          | City          | Receipts_sum |
| ------------------------------------------------------ | ------------- | ------------ |
| BJ'S RESTAURANT AND BREWHOUSE (5207 BRODIE LN STE 300) | SUNSET VALLEY |       79,361 |
| DOC'S BACKYARD (5207 BRODIE LN # 100)                  | SUNSET VALLEY |       58,080 |
| CANE ROSSO (4715 S LAMAR BLVD STE 103)                 | SUNSET VALLEY |       31,824 |


In [32]:
location_sum('City', 'WEST LAKE HILLS').limit(3).print_table(max_column_width=60)

| Establishment                                                | City            | Receipts_sum |
| ------------------------------------------------------------ | --------------- | ------------ |
| LUPE TORTILLA MEXICAN RESTAURANT (701 S CAPITAL OF TEXAS ... | WEST LAKE HILLS |       91,059 |
| CHIPOTLE CHIPOTLE MEXICAN GRILL CHIPOTLE BURRITOS (3300 B... | WEST LAKE HILLS |          480 |


## Sales by county
In this case, we pass in the location type of 'County' and then a county name in caps to get the most sales in a particular county.

In [33]:
# MOVE TO SEPARATE CELLS SO THEY DON'T FORCE A SCROLL
location_sum('County', 'BASTROP').limit(5).print_table(max_column_width=80)
print('\n')
location_sum('County', 'CALDWELL').limit(5).print_table(max_column_width=80)
print('\n')
location_sum('County', 'HAYS').limit(5).print_table(max_column_width=80)
print('\n')
location_sum('County', 'TRAVIS').limit(5).print_table(max_column_width=80)
print('\n')
location_sum('County', 'WILLIAMSON').limit(5).print_table(max_column_width=80)

| Establishment                                                   | County  | Receipts_sum |
| --------------------------------------------------------------- | ------- | ------------ |
| LOST PINES BEVERAGE LLC (575 HYATT LOST PINES ROAD)             | Bastrop |      290,195 |
| OLD TOWN RESTURANT AND BAR/PINEY CREEK CHOP HOUSE (931 MAIN ST) | Bastrop |       94,519 |
| SP BASTROP THEATRE, LP (1600 CHESTNUT ST)                       | Bastrop |       43,273 |
| CHILI'S GRILL & BAR (734 HIGHWAY 71 W)                          | Bastrop |       42,059 |
| BACK 9 (834 HIGHWAY 71 W)                                       | Bastrop |       27,018 |


| Establishment                                               | County   | Receipts_sum |
| ----------------------------------------------------------- | -------- | ------------ |
| EL REY BAR/NIGHTCLUB (1721 S COMMERCE ST)                   | Caldwell |       22,083 |
| GUADALAJARA MEXICAN RESTAURANT (1710 S COLORADO ST STE 110) | Caldwell |   

## Sales by ZIP Code
A list of sales by ZIP Code. If anything other than 78701 is at the top, it is news.

In [34]:
# top zip code gross receipts
zip_receipts = mixbev.pivot('Zip', aggregation=agate.Sum('Receipts')).order_by('Sum', reverse=True)
zip_receipts.limit(5).print_table()

| Zip   |        Sum |
| ----- | ---------- |
| 78701 | 23,456,454 |
| 75201 | 12,523,621 |
| 77002 | 12,056,047 |
| 78205 | 11,913,671 |
| 77024 |  8,417,423 |
